In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
import pyspark
from pyspark.sql import SparkSession

In [ ]:
sc = SparkContext('local', 'Spark SQL') 

In [ ]:
sc.version

'2.4.7'

In [ ]:
sqlcon = SQLContext(sc)

In [ ]:
df2 = sqlcon.read.option("header",False) \
     .option("inferScema", True) \
     .csv("hdfs:///ass1/pullreq_events.csv") \

In [ ]:
df3 = df2.withColumnRenamed("_c0","pull_requestid") \
    .withColumnRenamed("_c1","author") \
    .withColumnRenamed("_c2", "event") \
    .withColumnRenamed("_c3", "time")

In [ ]:
df3.createTempView("pr")

In [ ]:
%%time
sqlcon.sql(" select a.name,b.week,b.countscore \
from(select hello.name, max(events) even, hello.week \
from (select author as name,date_trunc('week',time) week, count(event) events \
from pr where event='discussed' \
group by date_trunc('week',time), author) as hello \
group by hello.week,hello.name \
order by week,even desc)as a, \
(select hello.week, max(events) countscore \
from (select date_trunc('week',time) week, count(event) events \
from pr where event='discussed' \
group by date_trunc('week',time), author) as hello \
group by hello.week)as b \
where a.week=b.week and a.even=b.countscore").show()

+------------+-------------------+----------+
|        name|               week|countscore|
+------------+-------------------+----------+
|       mikel|2010-09-06 00:00:00|        16|
|       mikel|2010-09-13 00:00:00|         6|
|   josevalim|2010-09-20 00:00:00|         9|
|   josevalim|2010-09-27 00:00:00|         6|
|   josevalim|2010-10-04 00:00:00|        12|
|   josevalim|2010-10-11 00:00:00|         6|
|    krekoten|2010-10-18 00:00:00|         4|
|         fxn|2010-10-25 00:00:00|         1|
|  spastorino|2010-11-01 00:00:00|         2|
|        rsim|2010-11-01 00:00:00|         2|
|   josevalim|2010-11-08 00:00:00|         6|
|franckverrot|2010-11-15 00:00:00|         3|
|   josevalim|2010-11-15 00:00:00|         3|
|  tenderlove|2010-11-22 00:00:00|         6|
|   josevalim|2010-11-29 00:00:00|         4|
|      drogus|2010-12-06 00:00:00|         2|
|   josevalim|2010-12-13 00:00:00|         7|
|         dhh|2010-12-20 00:00:00|         5|
|      jeremy|2011-01-03 00:00:00|

In [ ]:
%%time
sqlcon.sql("select a.name,b.monthnumber,b.countscore from(select hello.name, max(events) even, hello.monthnumber   from (select author as name,EXTRACT(MONTH from time) monthnumber, count(event) events   from pr where event='discussed'  group by EXTRACT(MONTH from time), author) as hello   group by hello.monthnumber,hello.name  order by monthnumber,even desc)as a,  (select hello.monthnumber, max(events) countscore   from (select EXTRACT(MONTH from time) monthnumber, count(event) events   from pr where event='discussed'  group by EXTRACT(MONTH from time), author) as hello  group by hello.monthnumber)as b  where a.monthnumber=b.monthnumber and a.even=b.countscore").show()

+-------------------+---------------------+
|          weekstamp|count(pull_requestid)|
+-------------------+---------------------+
|2010-08-30 00:00:00|                    2|
|2010-09-06 00:00:00|                   15|
|2010-09-13 00:00:00|                   17|
|2010-09-20 00:00:00|                   17|
|2010-09-27 00:00:00|                   13|
|2010-10-04 00:00:00|                   10|
|2010-10-11 00:00:00|                    5|
|2010-10-18 00:00:00|                    5|
|2010-10-25 00:00:00|                    3|
|2010-11-01 00:00:00|                    4|
|2010-11-08 00:00:00|                    9|
|2010-11-15 00:00:00|                    8|
|2010-11-22 00:00:00|                    9|
|2010-11-29 00:00:00|                    6|
|2010-12-06 00:00:00|                    5|
|2010-12-13 00:00:00|                    6|
|2010-12-20 00:00:00|                    7|
|2010-12-27 00:00:00|                    4|
|2011-01-03 00:00:00|                    9|
|2011-01-10 00:00:00|           

In [ ]:
%%time
sqlcon.sql("select author, count(event) events from pr where event='opened' and extract(year from time)=2011 group by author order by events desc limit 1").show()

In [ ]:
%%time
sqlcon.sql("select date(time), count(pull_requestId) from pr group by date(time) order by date(time)").show()

In [ ]:
%%time
sqlcon.sql("select to_char(time, 'MM-YYYY') mon, count(event) from pr where event='merged' and extract(year from time)=2010 group by mon").show()

In [ ]:
%%time
sqlcon.sql("select event,date(time),count(*) from pr group by event,date(time) having event='opened' order by date(time)").show()

In [ ]:
%%time
sqlcon.sql("select event,date(time),count(*) from pr group by event,date(time) having event='discussed' order by date(time)").show()